### Load the data

In [6]:
import os
import mlflow
import numpy as np

os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"
mlflow.set_experiment('Training_LSTM')

<Experiment: artifact_location='mlflow-artifacts:/976886767624453903', creation_time=1698104820917, experiment_id='976886767624453903', last_update_time=1698104820917, lifecycle_stage='active', name='Training_LSTM', tags={}>

In [7]:
# Create sequences of data to be used for training
def create_sequences(data, sequence_length):
    sequences = []
    target = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i+sequence_length])
        target.append(data[i+sequence_length])
    return np.array(sequences), np.array(target)

### Create the Optuna Objective

In [28]:
import keras.backend as K

def mean_absolute_percentage_error(y_true, y_pred):
    diff = K.abs((y_true - y_pred) / K.clip(K.abs(y_true), K.epsilon(), None))
    return 100.0 * K.mean(diff, axis=-1)

In [31]:
import optuna
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import mlflow.keras
from sklearn.preprocessing import MinMaxScaler

def objective(trial, data):
    with mlflow.start_run() as run:
        mlflow.keras.autolog(log_models=False)

        # Define the search space for hyperparameters
        num_layers = trial.suggest_int('num_layers', 1, 4)
        units_per_layer = [trial.suggest_int(f'units_layer_{i}', 32, 256, 32) for i in range(num_layers)]
        sequence_length = trial.suggest_int('sequence_length', 5, 20)
        learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-3)
        dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
        min_max_scaling = trial.suggest_int('min_max_scaling', 0, 1)

        mlflow.log_params({
            'num_layers': num_layers,
            'units_per_layer': units_per_layer,
            'sequence_length': sequence_length,
            'learning_rate': learning_rate,
            'dropout_rate': dropout_rate,
            'min_max_scaling': min_max_scaling
        })

        if min_max_scaling == 1:
            scaler = MinMaxScaler()
            data = scaler.fit_transform(np.array(data))

        X, y = create_sequences(data, sequence_length)

        # Build and compile the LSTM model
        model = keras.Sequential()
        for units in units_per_layer:
            model.add(keras.layers.LSTM(units, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)))
            model.add(keras.layers.Dropout(dropout_rate))
        model.add(keras.layers.Dense(1))

        optimizer = keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=1.0)
        model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[mean_absolute_percentage_error])
        # Split the data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

        # Define an EarlyStopping callback for preemptive pruning
        early_stopping = EarlyStopping(monitor='val_mean_absolute_percentage_error', patience=10)  # You can adjust the patience value

        # Train the model with early stopping
        history = model.fit(
            X_train,
            y_train,
            epochs=100,
            batch_size=32,
            validation_data=(X_val, y_val), 
            verbose=1,
            callbacks=[early_stopping]
        )

        # Evaluate the model on the validation set
        loss = model.evaluate(X_val, y_val)

        return loss

### Run the optimization

In [33]:
import numpy as np
from common import get_dataframe

# Normalize the data to the range [0, 1] to help the LSTM model converge faster

df = get_dataframe()

for coin in df.iloc[:, 1:]:
    data = np.array(df[coin]).reshape(-1, 1)

    # Create an Optuna study
    study = optuna.create_study(study_name=coin, direction='minimize')

    # Start the optimization process
    study.optimize(lambda trial: objective(trial, data), n_trials=100)

    # Get the best hyperparameters
    best_params = study.best_params
    print("Best Hyperparameters:", best_params)

[I 2023-10-24 01:56:13,214] A new study created in memory with name: close_NULSUSDT
2023/10/24 01:56:13 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: API request to endpoint /api/2.0/mlflow/runs/log-inputs failed with error code 404 != 200. Response body: '<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
'


Epoch 1/100
44/44 [==============================] - 2s 29ms/step - loss: 0.0401 - mean_absolute_percentage_error: 104681.9219 - val_loss: 0.0023 - val_mean_absolute_percentage_error: 69.0085
Epoch 2/100
44/44 [==============================] - 1s 18ms/step - loss: 0.0152 - mean_absolute_percentage_error: 66315.6406 - val_loss: 0.0016 - val_mean_absolute_percentage_error: 52.4860
Epoch 3/100
44/44 [==============================] - 1s 17ms/step - loss: 0.0122 - mean_absolute_percentage_error: 82103.0625 - val_loss: 0.0013 - val_mean_absolute_percentage_error: 46.2893
Epoch 4/100
44/44 [==============================] - 1s 17ms/step - loss: 0.0106 - mean_absolute_percentage_error: 89969.7031 - val_loss: 0.0011 - val_mean_absolute_percentage_error: 39.0257
Epoch 5/100
44/44 [==============================] - 1s 16ms/step - loss: 0.0093 - mean_absolute_percentage_error: 84250.9375 - val_loss: 0.0011 - val_mean_absolute_percentage_error: 40.4325
Epoch 6/100
44/44 [=========================

[W 2023-10-24 01:56:49,251] Trial 0 failed with parameters: {'num_layers': 1, 'units_layer_0': 192, 'sequence_length': 16, 'learning_rate': 0.0005126656520215088, 'dropout_rate': 0.0767748130030077, 'min_max_scaling': 1} because of the following error: The number of the values 2 did not match the number of the objectives 1.
[W 2023-10-24 01:56:49,251] Trial 0 failed with value [0.0005226924549788237, 29.448570251464844].
2023/10/24 01:56:49 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: API request to endpoint /api/2.0/mlflow/runs/log-inputs failed with error code 404 != 200. Response body: '<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
'


Epoch 1/100
44/44 [==============================] - 2s 18ms/step - loss: 0.1422 - mean_absolute_percentage_error: 65.5508 - val_loss: 0.0155 - val_mean_absolute_percentage_error: 49.7345
Epoch 2/100
44/44 [==============================] - 1s 12ms/step - loss: 0.0602 - mean_absolute_percentage_error: 36.6820 - val_loss: 0.0058 - val_mean_absolute_percentage_error: 31.4443
Epoch 3/100
44/44 [==============================] - 1s 13ms/step - loss: 0.0403 - mean_absolute_percentage_error: 28.0612 - val_loss: 0.0039 - val_mean_absolute_percentage_error: 24.8754
Epoch 4/100
44/44 [==============================] - 1s 13ms/step - loss: 0.0325 - mean_absolute_percentage_error: 25.5670 - val_loss: 0.0035 - val_mean_absolute_percentage_error: 24.1759
Epoch 5/100
44/44 [==============================] - 1s 12ms/step - loss: 0.0267 - mean_absolute_percentage_error: 23.3257 - val_loss: 0.0030 - val_mean_absolute_percentage_error: 21.9520
Epoch 6/100
44/44 [==============================] - 1s 14ms

[W 2023-10-24 01:57:45,171] Trial 1 failed with parameters: {'num_layers': 1, 'units_layer_0': 256, 'sequence_length': 8, 'learning_rate': 0.0004220188479990986, 'dropout_rate': 0.3407961854115491, 'min_max_scaling': 0} because of the following error: The number of the values 2 did not match the number of the objectives 1.
[W 2023-10-24 01:57:45,172] Trial 1 failed with value [0.00031987534021027386, 6.3033246994018555].
2023/10/24 01:57:45 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: API request to endpoint /api/2.0/mlflow/runs/log-inputs failed with error code 404 != 200. Response body: '<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
'


Epoch 1/100
44/44 [==============================] - 5s 56ms/step - loss: 0.0450 - mean_absolute_percentage_error: 139783.4688 - val_loss: 0.0091 - val_mean_absolute_percentage_error: 133.5536
Epoch 2/100
44/44 [==============================] - 2s 47ms/step - loss: 0.0227 - mean_absolute_percentage_error: 99196.4766 - val_loss: 0.0029 - val_mean_absolute_percentage_error: 80.5042
Epoch 3/100
44/44 [==============================] - 2s 43ms/step - loss: 0.0168 - mean_absolute_percentage_error: 88685.7656 - val_loss: 0.0015 - val_mean_absolute_percentage_error: 52.3045
Epoch 4/100
44/44 [==============================] - 2s 42ms/step - loss: 0.0145 - mean_absolute_percentage_error: 86865.3281 - val_loss: 0.0014 - val_mean_absolute_percentage_error: 44.5386
Epoch 5/100
44/44 [==============================] - 2s 44ms/step - loss: 0.0130 - mean_absolute_percentage_error: 81513.5859 - val_loss: 0.0013 - val_mean_absolute_percentage_error: 42.4099
Epoch 6/100
44/44 [========================

### Load the best model configuration and train it

In [32]:

# Train the final model with the best hyperparameters
# Define the search space for hyperparameters
num_layers = best_params['num_layers']
units_per_layer = [best_params[f'units_layer_{i}'] for i in range(num_layers)]
sequence_length = best_params['sequence_length']
learning_rate = best_params['learning_rate']
dropout_rate = best_params['dropout_rate']

X, y = create_sequences(data, sequence_length)
_X_train, X_val, _y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(_X_train, _y_train, test_size=0.2, shuffle=False)

# Build and compile the LSTM model
best_model = keras.Sequential()
for units in units_per_layer:
    best_model.add(keras.layers.LSTM(units, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)))
    best_model.add(keras.layers.Dropout(dropout_rate))
best_model.add(keras.layers.LSTM(units_per_layer[-1], activation='relu'))
best_model.add(keras.layers.Dropout(dropout_rate))
best_model.add(keras.layers.Dense(1))

optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
best_model.compile(optimizer=optimizer, loss='mean_squared_error')

best_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

NameError: name 'best_params' is not defined

In [ ]:
from common import register_training_experiment
preds = best_model.predict([X_test])
register_training_experiment(y_test, preds)

#### Multivariate Time Series Forecasting with Deep Learning

[Source 1](https://towardsdatascience.com/multivariate-time-series-forecasting-with-deep-learning-3e7b3e2d2bcf) \
[Source 2](https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/)

In [ ]:
from common import get_clustered_dataframes

clusters_data = get_clustered_dataframes()

In [ ]:
clusters_data.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

def get_train_test_data(cluster_data, n_steps=10, test_size=0.2, shuffle=False):

    data = cluster_data.copy() 

    data['Date'] = pd.to_datetime(data['Date'])  # Convert the 'date' column to datetime

    # Sort the data by date
    data = data.sort_values(by='Date')

    # Create input data by shifting prices to create sequences
    X = data.drop(columns=['Date']).values
    X_seq = [X[i:i + n_steps] for i in range(len(X) - n_steps)]

    # Shift the closing price to predict the next closing price
    y = data.drop(columns=['Date']).shift(-n_steps).values

    # Split the data into training and testing sets
    _X_train, X_test, _y_train, y_test = train_test_split(X_seq, y[:-n_steps], test_size=0.2, shuffle=shuffle)

    # split the training set into training and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(_X_train, _y_train, test_size=0.2, shuffle=shuffle)

    # Reshape the data to 3D for LSTM
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    X_valid = np.array(X_valid)
    y_valid = np.array(y_valid)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    print('X_train shape:', X_train.shape)
    print('y_train shape:', y_train.shape)
    print('X_valid shape:', X_valid.shape)
    print('y_valid shape:', y_valid.shape)
    print('X_test shape:', X_test.shape)
    print('y_test shape:', y_test.shape)
    print("\n\n")

    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import mlflow


def calculate_rmse(y_true, y_pred):
    return sqrt(mean_squared_error(y_true, y_pred))


def build_model(n_steps, n_features):
    return tf.keras.Sequential([
        tf.keras.layers.LSTM(64, activation='relu', input_shape=(n_steps, n_features), return_sequences=True),
        tf.keras.layers.LSTM(64, activation='relu'),
        tf.keras.layers.Dense(n_features),
        tf.keras.layers.Dense(n_features)
    ])


def lstm_training(cluster_data, n_steps=10, test_size=0.2, shuffle=False):

    data = cluster_data.copy()
    X_train, X_valid, X_test, y_train, y_valid, y_test = get_train_test_data(data, n_steps, test_size, shuffle)
    
    model = build_model(n_steps, X_train.shape[2])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=250, batch_size=32, validation_data=(X_valid, y_valid))

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate and print RMSE
    for i, cripto in enumerate(data.columns[:-1]):
        rmse = calculate_rmse(y_test[:, i], y_pred[:, i])
        print(f'Root Mean Squared Error (RMSE) for {cripto}: {rmse:.4f}')

In [ ]:
for cluster, cripto in clusters_data.items():
    print(f'Cluster {cluster}: {cripto.columns[:-1]}\n')

    lstm_training(cripto, n_steps=2)

    print("\n---------------------------------\n")